<a href="https://colab.research.google.com/github/EdmarBortoletto/Ciencias-de-Dados/blob/main/Analise_Sentimentos_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from google.colab import files
import numpy as np

In [ ]:
!pip install keras.utils


  Preparing metadata (setup.py) ... done
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2632 sha256=fa5e1224ed53d4ea376149689a3e220909520d9420a8e4a25ff7301a731ba854
  Stored in directory: /root/.cache/pip/wheels/5c/c0/b3/0c332de4fd71f3733ea6d61697464b7ae4b2b5ff0300e6ca7a
Successfully built keras.utils


In [ ]:
from sklearn.preprocessing import LabelEncoder



In [ ]:
!pip install keras.preprocessing


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 875.2 kB/s eta 0:00:00


In [ ]:
from keras_preprocessing.sequence import pad_sequences #Tokens com mesmo tamanho

In [ ]:
files.upload()


In [ ]:
data = pd.read_csv("tw_pt.csv")
data.head()

,Unnamed: 0,Text,Classificacao
0,0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,3,��� https://t.co/BnDsO34qK0,Neutro
4,4,��� PSOL vai questionar aumento de vereadores ...,Negativo


In [ ]:
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])


##Analise exploratoria dos dados

In [ ]:
data.groupby(['Classificacao']).size()

Classificacao
Negativo    2446
Neutro      2453
Positivo    3300
dtype: int64

In [ ]:
#Filtragem
#pegar dados com maio confiança - Dados com maior qualidade
#data = data[data['']> 1.0]

In [ ]:
#Dados com forma numerica e tabular
token = Tokenizer(num_words=100)
token.fit_on_texts(data['Text'].values) #Artibuto onde esta o comentario do usuario

In [ ]:
#Gerar os tokens
#Completar tokens com 0 a direita(padronizar tamanho - tabulação)
X = token.texts_to_sequences(data['Text'].values)
X = pad_sequences(X,padding="post",maxlen=100)

In [ ]:
print(X)

[[ 1 29 70 ...  0  0  0]
 [29 70  6 ...  0  0  0]
 [29 70  6 ...  0  0  0]
 ...
 [16 49 85 ...  0  0  0]
 [16 49 85 ...  0  0  0]
 [85  1 42 ...  0  0  0]]


In [ ]:
#transformar as 3 classes em dado numérico
labelencoder = LabelEncoder()


In [ ]:
y = labelencoder.fit_transform(data['Classificacao'])
print(y)

[1 1 1 ... 2 2 2]


In [ ]:
#Classes de saida em formato numerico
y = to_categorical(y)
print(y)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)
X_test

array([[ 7, 63, 19, ...,  0,  0,  0],
       [ 7, 66, 67, ...,  0,  0,  0],
       [22, 10,  1, ...,  0,  0,  0],
       ...,
       [31, 47, 36, ...,  0,  0,  0],
       [ 7, 43,  5, ...,  0,  0,  0],
       [ 7,  5, 20, ...,  0,  0,  0]], dtype=int32)

In [ ]:
#empilhar camadas da RNP
modelo = Sequential()
modelo.add(Embedding(input_dim= len(token.word_index),output_dim=128,input_length=X.shape[1]))
modelo.add(SpatialDropout1D(0.2))
#Cria camada Deep Learning
 #units: neuronios

modelo.add(LSTM(units=196, dropout=0.2,recurrent_dropout=0,activation='tanh',
                 recurrent_activation='sigmoid',unroll=False,use_bias=True))
modelo.add(Dense(units=3,activation="softmax"))

In [ ]:
modelo.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(modelo.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1717120   
                                                                 
 spatial_dropout1d (Spatial  (None, 100, 128)          0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 1972511 (7.52 MB)
Trainable params: 1972511 (7.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#Treina a rede Neural
#batch_size:atualização
#validation_data: Valida performance com dados de treino - x e y(não usado no treino)
modelo.fit(X_train,y_train,epochs=10,batch_size=30,verbose=True,validation_data=(X_test,y_test))

Epoch 1/10
192/192 [==============================] - 64s 323ms/step - loss: 1.0893 - accuracy: 0.4065 - val_loss: 1.0928 - val_accuracy: 0.3927
Epoch 2/10
192/192 [==============================] - 49s 257ms/step - loss: 1.0877 - accuracy: 0.4067 - val_loss: 1.0912 - val_accuracy: 0.3927
Epoch 3/10
192/192 [==============================] - 55s 285ms/step - loss: 1.0880 - accuracy: 0.4067 - val_loss: 1.0920 - val_accuracy: 0.3927
Epoch 4/10
192/192 [==============================] - 51s 265ms/step - loss: 1.0880 - accuracy: 0.4067 - val_loss: 1.0913 - val_accuracy: 0.3927
Epoch 5/10
192/192 [==============================] - 51s 264ms/step - loss: 1.0877 - accuracy: 0.4067 - val_loss: 1.0909 - val_accuracy: 0.3927
Epoch 6/10
192/192 [==============================] - 51s 266ms/step - loss: 1.0880 - accuracy: 0.4067 - val_loss: 1.0913 - val_accuracy: 0.3927
Epoch 7/10
192/192 [==============================] - 50s 262ms/step - loss: 1.0875 - accuracy: 0.4067 - val_loss: 1.0916 - val_ac

In [ ]:
#Mini previsao para avaliar performance
loss,accuracy=modelo.evaluate(X_test,y_test)
print("Loss",loss)
print("Accuracy",accuracy)

77/77 [==============================] - 6s 70ms/step - loss: 1.0915 - accuracy: 0.3927
Loss 1.0915050506591797
Accuracy 0.3926829397678375


In [ ]:
#Realizando a previsao
prev = modelo.predict(X_test)
print(prev)

77/77 [==============================] - 6s 74ms/step
[[0.29362524 0.29726848 0.40910628]
 [0.29362524 0.29726848 0.40910628]
 [0.29362524 0.29726848 0.40910628]
 ...
 [0.29362524 0.29726848 0.40910628]
 [0.29362524 0.29726848 0.40910628]
 [0.29362524 0.29726848 0.40910628]]


## Algoritmo Vader

In [ ]:
from pandas.core.internals.managers import T
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
files.upload()

In [ ]:
data2 = pd.read_csv('tw_pt.csv')
data2.shape

(8199, 3)

In [ ]:
data2.head()

,Unnamed: 0,Text,Classificacao
0,0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,3,��� https://t.co/BnDsO34qK0,Neutro
4,4,��� PSOL vai questionar aumento de vereadores ...,Negativo


In [ ]:
data2.groupby(['Classificacao']).size()

Classificacao
Negativo    2446
Neutro      2453
Positivo    3300
dtype: int64

In [ ]:
data2.loc[data2['Classificacao'] == 'Irrelevante','Classificacao'] = 'Neutro'

In [ ]:
data2 = data2.dropna(subset=['Text'])
data2.reset_index(drop = True, inplace = True)

In [ ]:
data2.shape

(8199, 3)

In [ ]:
from textblob import TextBlob
teste = TextBlob("I Love NY!")
print(teste.sentiment)

Sentiment(polarity=0.625, subjectivity=0.6)


In [ ]:
!pip install translate

In [ ]:
from translate import Translator


In [ ]:
import nltk
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
mas = SentimentIntensityAnalyzer()

In [ ]:
data2['vander_Sentimento'] = ''

In [ ]:
for y in range(len(data2.index)):
  x = mas.polarity_scores(data2['Text'].iloc[y])
  del x['compound']
  maior = max(x,key=x.get) #neg pos neu
  data2.loc[y,'vander_Sentimento'] = maior

In [ ]:
data2.groupby(['vander_Sentimento']).size()

In [ ]:
data2.groupby(['Classificacao']).size()

Classificacao
Negativo    2446
Neutro      2453
Positivo    3300
dtype: int64

In [ ]:
data2.loc[data2['vander_Sentimento']=='neu' , 'vander_Sentimento'] = 'Neutro'
data2.loc[data2['vander_Sentimento']=='neg' , 'vander_Sentimento'] = 'Negativo'
data2.loc[data2['vander_Sentimento']=='pos' , 'vander_Sentimento'] = 'Positivo'

In [ ]:
data2.groupby(['vander_Sentimento']).size()

vander_Sentimento
Neutro      8198
Positivo       1
dtype: int64

In [ ]:
y_pred = data2['vander_Sentimento']
y_test = data2['Classificacao']
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[   0 2446    0]
 [   0 2452    1]
 [   0 3300    0]]


#Testando outras possibilidades


In [ ]:
previsoes = np.argmax(prev, axis=1)
previsoes

In [ ]:
hopihari['Sentimento'].unique()

In [ ]:
valores_unicos = hopihari['Sentimento'].unique()
print(valores_unicos)


In [ ]:
# Supondo que 'hopihari' é o seu DataFrame
coluna_sentimento = hopihari['Sentimento']
print(coluna_sentimento)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Matriz de previsões (substitua pela sua própria matriz)
prev = np.array([
    [0.07421661, 0.11202153, 0.8137619],
    [0.07421661, 0.11202153, 0.8137619],
    # ... outras previsões ...
])

# Nomes das classes
classes = ['Classe 0', 'Classe 1', 'Classe 2']

# Número de exemplos
num_exemplos = previsoes.shape[0]

# Criar um gráfico de barras empilhadas
fig, ax = plt.subplots()
ax.bar(range(num_exemplos), previsoes[:, 0], label=classes[0])
ax.bar(range(num_exemplos), previsoes[:, 1], bottom=previsoes[:, 0], label=classes[1])
ax.bar(range(num_exemplos), previsoes[:, 2], bottom=previsoes[:, 0] + previsoes[:, 1], label=classes[2])

# Configurar rótulos e legendas
ax.set_ylabel('Probabilidade')
ax.set_title('Previsões do Modelo')
ax.set_xticks(range(num_exemplos))
ax.set_xticklabels(range(1, num_exemplos + 1))
ax.legend()

plt.show()
